In [1]:
import os
from hashlib import sha256
from Crypto.Util.number import *
def xor(data1, data2):
    return bytes([data1[i] ^ data2[i] for i in range(len(data1))])

def do_round(data, key):
    m = sha256()
    m.update(xor(data[2:4], key))
    return bytes(data[2:4]) + xor(m.digest()[0:2], data[0:2])

def do_round_inv(data, key):
    m = sha256()
    m.update(xor(data[0:2], key))
    return xor(m.digest()[0:2], data[2:4]) + bytes(data[0:2])

def pad(data):
    padding_length = 4 - (len(data) % 4)
    return data + bytes([padding_length] * padding_length)

def unpad(data):
    padding_length = data[-1]
    return data[:-padding_length]

# XOR every character with bytes generated from the PRNG
def encrypt_block(data, key):
    
    for i in range(10):
        data = do_round(data, key)
    return data

def decrypt_block(data, key):
    for i in range(10):
        data = do_round_inv(data, key)
    return data

def encrypt_data(data, key):
    cipher = b''
    while data:
        cipher += encrypt_block(data[:4], key)
        data = data[4:]
    return cipher

def decrypt_data(cipher, key):
    data = b''
    while cipher:
        data += decrypt_block(cipher[:4], key)
        cipher = cipher[4:]
    return data

def encrypt(data, key):
    data = pad(data)
    return encrypt_data(encrypt_data(data, key[0:2]), key[2:4])

def decrypt(data, key):
    plain = decrypt_data(decrypt_data(data, key[2:4]), key[0:2])
    return unpad(plain)

In [2]:
from pwn import *
r = remote("brutalmogging.ctf.umasscybersec.org","1337")
print(r.recvuntil("s:"))
t1=r.recvline().strip()
r.close()

[x] Opening connection to brutalmogging.ctf.umasscybersec.org on port 1337
[x] Opening connection to brutalmogging.ctf.umasscybersec.org on port 1337: Trying 35.231.243.29
[+] Opening connection to brutalmogging.ctf.umasscybersec.org on port 1337: Done


/tmp/ipykernel_1937/102102624.py:3: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  print(r.recvuntil("s:"))


b"== proof-of-work: disabled ==\nOh yeah, my cipher is so strong and my one way function is so well defined.\nNo betas can ever break it, so I'll just give you the flag right now.\nThe encrypted flag is:"
[*] Closed connection to brutalmogging.ctf.umasscybersec.org port 1337


In [3]:
t1

'7213d06923a531a4db074b823ba8de37512553077ac5d9742fc13543ec4fc68aba0a8ab27dbacc29d82f731d946773d8'

In [4]:
from Crypto.Util.number import *
a1 =[] 
for i in range(256):
    a1.append(long_to_bytes(i))
partial_keys=[]
for i in a1:
    for j in a1:
        partial_keys.append(i+j)

In [5]:
len(partial_keys)

65536

In [6]:
h1=[]
for i in partial_keys:
    h1.append(decrypt_data(long_to_bytes(int(t1[:8],16)),i))

In [8]:
h2=[]
for i in partial_keys:
    h2.append(encrypt_data(b'UMAS',i))

In [10]:
len(h1),len(h2)

(65536, 65536)

In [11]:
g1=list(set(h1).intersection(h2))[0]

In [12]:
g1

b'1\xb5\x14?'

In [14]:
key = partial_keys[h2.index(g1)] + partial_keys[h1.index(g1)]

In [15]:
key

b'\xcf\xbfJ9'

In [16]:
decrypt(long_to_bytes(int(t1,16)),key)

b'UMASS{1_h4ve_b33n_m3w1ng_f0r_my_l1f3_733061741}'